# Time series analysis workshop 


## Guided notebook

This is the guided notebook for the time series analysis workshop. You will find the code for the visualisations and analysis we will carry out in class in the completed notebook. Feel free to follow along or review your answers after the workshop.



### Data

The data we will be using can be downloaded [here](https://fred.stlouisfed.org/series/DEUPROINDMISMEI) from the FRED database, the Federal Bank of St. Louis's economic database. You can download the data as a `.csv` file by clicking on the `download` button in the top right corner. 

This dataset shows monthly values of seasonally-adjusted total industrial production in Germany between January 1960, and October 2020. The data each month is expressed as a percentage of the value of industrial production in Germany in 2015.

Once you download the dataset, make sure to save it in the same directory as this notebook so that you can read in the data without errors.

**Important**: We will use this data to fit and forecast an ARIMA model. ARIMA is particularly well suited for data with little or no seasonality, such as seasonally adjusted series. We will discuss this further in the online practice.

### Goal

Our goal today is to underestand the properties of our time series, and to perform the necessary transformations needed to produce a simple forecast using an ARIMA model.

### Agenda

Today we will:

* plot the time series


* perform a decomposition


* test for stationarity


* perform differencing


* discuss and implement an `ARIMA(p,d,q)` model

### Preliminary steps 

These are the steps we will follow for our analysis. 

1. Load the data 

2. Explore and clean

3. Decompose the series and choose the right type of decomposition

4. Apply `log transformation` to the series if you choose a multiplicative decomposition

Remember: log transformation serves the purpose of making sure that the variance of our series is constant over time. This makes prediction easier. This will change the interpretation of the units of our series, so we will need to transform it back when we produce forecasts to interpret. We do this by performing the inverse operation: `exponential`.

### Modelling steps: prepare the data for an ARIMA

This data seems to be particularly well suited for an ARIMA model, given that ARIMA requires a series to be **univariate**: only one variable, industrial production, is observed over time. Remember that we want time series data to be **evenly spaced**: to be observed at regular time intervals. This is true for this series, because that is the way this data is collected, and you can check it yourself once we import it. 

ARIMA also requires the data to be **stationary**, so we will run tests to assess whether that is the case, and if it is not we will apply differencing to make it so. 

1. Plot the ACF/PACF and look for evidence of trends and/or seasonality

2. Run the ADF and KPSS tests - most likely our series will not be stationary!

3. Perform differencing 

4. Run the ADF and KPSS tests again

5. Plot the ACF/PACF again to assess whether there are any remaining patterns

6. Run the ADF and KPSS tests again 

7. Repeat steps 3-6 until the data is stationary

8. Plot the ACF and PACF plots of the **stationary series**

9. Select a value for `p` and `q` from the PACF and ACF plots respectively. `d` will be the number of times you differenced the series to make it stationary

10. Split the data into train and test set

10. Run `ARIMA(p,d,q)` using `ARIMA()` from the `statsmodels` package. **Note**: you will need to input your series **before differencing** as an input here, as this function performs the differencing for you. 

11. Check residuals for autocorrelation

12. Produce forecasts and compare to the test set

13. Transform the forecasts back from log to level 

### Imports

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose #we will need this 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, kpss
import pmdarima as pm

import warnings
warnings.filterwarnings("ignore")

### Load the data

You can download the `csv` file from this [link](https://fred.stlouisfed.org/series/DEUPROINDMISMEI). Make sure you save it in the same directory as this notebook. Feel free to rename it to something more intuitive to you.

In [3]:
# save the .csv file from the FRED website and load it using a pandas in-built method


### Explore and clean

In [1]:
# print the head of the df. What is the frequency of our data? What is the first date available?



In [2]:
# print the tail of the df - the last day was September 1st 2020



In [3]:
# shape of the df



In [4]:
# explore datatypes and count of values in each column



In [8]:
# transform the date column to datetime datatype



In [9]:
# index the data by date



In [10]:
# rename the column we are interested in to something more humanly legible



In [12]:
# drop date from axis



The series goes back in time to the '60s, so we are working with quite a large dataset. Given the nature and composition of industrial production has changed significantly since the 1960's, we can limit our datasets to observations occurring after 1990. 

In [15]:
# only keep observations indexed with year=1990 or higher



In [5]:
# plot the series



### Decomposition

You can test your intuition visually recognising multiplicative/additive decomposition series [here](https://kourentzes.com/forecasting/2014/11/09/additive-and-multiplicative-seasonality/)

In [16]:
# produce both additive and multiplicative decompositions




In [6]:
# plot the two decompositions against the original data



These two plots compare the deterministic parts of the series (trend and seasonal components combined) produced with a multiplicative and additive model (red and green series respectively), to the actual time series (grey series). 

Sometimes it is visually easy to detect which decomposition model suits our data best, in this case it is not, so we will need to compare decomposition residuals. The model with the highest residuals is the worse performing one. 

In [7]:
# compare residuals from the two decompositions using the .resid method




From this plot, there is not much difference when it comes to the average value of the series, but it does seem that the spread of values in the additive decomposition is much higher (look at the y axis of both plots), so we will pick a **multiplicative decomposition**.

In [8]:
# run seasonal_decompose model="multiplicative"



In [9]:
# plot just the trend component(.trend) of the series



In [10]:
# plot just the residual component(.resid) of the series




In [11]:
# plot just the seasonal component(.seasonal) of the series



### Log transformation

When we decide a multiplicative decomposition is best for our series it is generally a good idea to apply a `log transformation` to it. By doing so, we effectively transform the series into an additive decomposition series of log values. 

This is true because, as we saw in the introduction the deterministic part of a series `Y` in context of a multiplicative decomposition will be obtained like this: `T`x`S`, where T is the trend component, and S the seasonal one. If we apply a log transformation to this product, we get find that the `log(Y) = log(T) + log(S)`. If you are interested you can read more about logarithm algebra [here](https://courses.lumenlearning.com/boundless-algebra/chapter/working-with-logarithms/#:~:text=produce%20that%20number.-,The%20logarithm%20of%20a%20product%20is%20the%20sum%20of%20the,the%20difference%20of%20the%20logarithms).

We need to do this because multiplicative series generally have **increasing variance** as the seasonal fluctuations increase in time. Instead, we need our data to have constant variance, so that we can make better predictions. 


In [46]:
# log transform the series using np.log() and assign to the old variable



## Modelling: preparing data for ARIMA

### Plot the ACF and PACF

In [12]:
# plot the acf using plot_acf()



In [13]:
# plot the pacf using plot_pacf()



We can clearly observe the presence of a trend, while it is unclear that there is any seasonality. However, the presence of a trend tells us that the series is **not stationary**

### Stationarity: ADF test

The ADF test for stationarity.

H<sub>o</sub>: there is a unit root

H<sub>1</sub>: there is no unit root, the series is stationary
    
significance level (alpha): 0.05

This means that if you manage to reject the null hypothesis, `p-value < alpha`, your series is stationary


In [49]:
#stationarity - adf test, run using adfuller() and set autolag='AIC'



In [15]:
# compare the p-value of the test to 0.05. The p-value is the second output of the test



### Stationarity: the KPSS test

KPSS test for stationarity:

H<sub>o</sub>: there is no unit root, the series is stationary

H<sub>1</sub>: there is a unit root

This means that if you manage to reject the null hypothesis, `p-value < alpha`, your series is NOT stationary

In [51]:
# run the kpss test using the kpss() function


In [16]:
# print the p-value and compare to 0.05. Once again p-value is the second output of the test


### Differencing

Remember that we need our series to be stationary in order to apply an ARIMA model to produce forecasts. The most common way to do that is through "differencing". We will subtract consecutive observations in order to remove trend and any seasonality in the series: we assume that increases in trend for consecutive periods are roughly constant, so differencing tends to be effective at removing trends.

In this workshop we will discussing the case when a series has a trend, but not strong seasonality (as we have seen from the ACF/PACF plots in this industrial production data).

**Important: differencing once makes a series stationary when there is a linear trend. Differencing twice makes a series stationary when there is a quadratic trend.**

In the online practice we will see that simple differencing is not always enough to remove both trend and seasonality - in that case we will resort to "seasonal differencing".

In [54]:
# differencing -1st order to eliminate the trend 
# drop na's resulting from differencing



In [17]:
#let's visualise the 1st differenced series



### Run ADF and KPSS tests again

In [18]:
# run adf test again and compare p-value to alpha



In [19]:
# run kpss test again
# compare p-value to alpha



In [20]:
# plot acf of the differenced series to check for any trend/seasonality left



## Implement ARIMA

Implementing ARIMA requires a set of assumptions to hold:

- the data should be **stationary**, in other words its properties should not vary over time. This means that its mean and variance should be constant (homoskedastic residuals), which is why we had to apply differencing to make our series stationary. 

- the series should be **univariate**, that is be composed of only one variable. There are no other predictors (e.g. in an extension to ARIMA, SARIMAX, you will see that there are exceptions to this rule)

We verified that our series is stationary when differenced once, and that we only have one variable, `industrial production` in this dataset.  

**ARIMA is composed of three hyperparameters, which we need to input when running the `ARIMA()` function**:

- `p`: number of autoregressive terms (AR order)
- `d`: number of nonseasonal differences (differencing order) 
- `q`: number of moving-average terms (MA order)

**Important**: `ARIMA()` in `statsmodels` will perform the differencing for us, so we need to do it manually beforehand to understand what level of differencing is required for stationarity, in order to specify the `d` parameter when running the model. 

*However, your input series will be the non-differenced one as the function will do the differencing for you, you just need to specify how many differences are necessary for stationarity.*


`p` and `q` are chosen by looking at the PACF and ACF plots respectively. They are chosen by looking at the lag after which the autocorrelations become statistically insignificant. 

**Important**: we will be looking at the PACF/ACF plots of the **stationary series**, in other words, the differenced one. 

**Let's have a look at how all this works in practice**

### Plot the ACF/PACF of the stationary series

The first step is to plot the ACF and PACF plots of the stationary series, so that we can choose a value for the `p` and `q` parameters from the PACF and ACF plots respectively

In [21]:
# plot acf of the differenced series



Let's take a look at this ACF plot to choose the best `q` value. 

In [22]:
# plot pacf of the differenced series



Let's take a look at this ACF plot to choose the best `p` value. 

Having looked at ACF, PACF plots and having carried out differencing to achieve stationarity we can set our initial parameters to:

- `p`: 2 (PACF plot)
- `d`: 1 (as we had to difference the series once to make it stationary)
- `q`: 2 (ACF plot)

### Train-test split

In [23]:
# produce a train test split

#split into train and test set - suggested 80-20 split, but you can choose your own split


# Check size


### Fit the ARIMA(p,d,q) model

Now that we found our model parameters, let's run the first model, fit it and analyse it. Notice that we are using our original log transformed series and not the differenced one

In [24]:
# run and fit our first model, print model summary()



In [25]:
# plot our fit
# dynamic = False: If dynamic is False, then the in-sample lagged values are used for prediction.
# If `dynamic` is True, then in-sample forecasts are used in place of lagged dependent variables. 



### Check residuals

An important part of analysing how good a model is relies on checking residuals. In general, we want them to:

- behave like white noise, and not show signs of patterns
- not have autocorrelation
- have a mean of 0, and have a symmetric distribution around the mean

If these conditions are not met, there is likely some pattern in our data that our model is not capturing. 

In [26]:
# check residuals of the model fit



### Forecast - model 1

Now that we have our model, let's use it to produce in-sample forecasts: forecast the values of the test set, which we will then compare to the actual values to evaluate performance. 

In [27]:
# produce forecasts and confidence intervals using the model we fit above



In [29]:
# transform the forecasts and upper/lower bounds of the confidence intervals to pd.Series()



In [30]:
# plot the different series: forecast series and test set values



### Transform the series back to level

Now that we have produced forecasts, let's revert the series back to level (reverse the log transformation). We can reverse a log transformation by applying its inverse operation: the exponential. The series will look exactly the same, only the units on the y axis will change

In [31]:
# apply np.exp() to the series before plotting the forecasts



### Optional stretch

We have run an ARIMA model, but we have not discussed how to select the best "version" of this model, or more formally, the best "specification". We will discuss this further in the online practice, but we do this using the `AIC` criteria. `AIC` stands for `Akaike Information Criteria`.

We ideally want to select the version of the model with the lowest AIC possible. 

We can do this automatically using techniques such as `gridsearch`, or in-built functions such as `autoarima`, which was a famous R function. Let's try the second: we will find an alternative model to the one above using autoarima, and compare the AIC we obtain with the previous one.

In [ ]:
# run a model using pm.auto_arima()



In [ ]:
# run and fit the new model 



In [ ]:
# compare the AIC value you obtain now with the previous model's. What do you conclude?



### Next steps

In the online practice we will discuss ways to find the optimal model, including automated techniques such as grid search. We will also discuss model evaluation techniques.